In [1]:
def div_poly(F, f, R = R):
    """
    Implementa el algoritmo de division
    de polinomios multivariados.
    De: Ideals, Varieties, and Algorithms, pag 65
    F es una lista de polinomios, f es el polinomio a dividir
    R es el conjunto de polinomios sobre el que se
    """
    q = [R(0) for _ in range(len(F))]
    r = R(0)
    p = f

    while p!=R(0):
        i=0
        divisionoccurred = False
        while i<len(F) and not divisionoccurred:

            quo = R.monomial_quotient(p.lt(),F[i].lt(),coeff=True)
            rem = p.lt().reduce(F[i].lt())

            if rem == 0:
                q[i] = q[i]+quo
                p = p - quo*F[i]
                divisionoccurred = True
            else:
                i += 1
        if not divisionoccurred:
            r = r+p.lt()
            p = p-p.lt()

    return q, r

In [2]:
def is_Groebner(F):
    """
    Revisa si una lista de polinomios F es una base de Groebner usando el
    Criterio de Buchberger.
    
    INPUT:
    - F: lista de polinomios en un anillo multivariable
    
    OUTPUT:
    - True si F es base de Groebner, False si no lo es
    """
    # Manejar caso de lista vacía o ideal cero
    if not F:
        return True
    
    # Revisar todos los pares (i, j) con i < j
    for i in range(len(F)):
        for j in range(i + 1, len(F)):
            # Saltamos si alguno de los polinomios es cero
            if F[i] == 0 or F[j] == 0:
                continue
                
            # Calcular el S-polinomio
            S_poly = S_polynomial(F[i], F[j])
            
            # Calcular el residuo de la división por F
            # No nos importa el cociente (el _), solo el residuo
            _, remainder = division_algorithm(S_poly,F)
            
            # Si el residuo es distinto de cero, falló el criterio: NO es base de Groebner
            if remainder != 0:
                return False
    
    # Si todos los S-polinomios reducen a cero, entonces sí es base
    return True


def division_algorithm(f, F):
    """
    Implementación del algoritmo de división del §3 (Teorema 3).
    
    INPUT:
    - f: polinomio a dividir
    - F: s-tupla ordenada de polinomios (f_1, ..., f_s)
    
    OUTPUT:
    - q: lista de cocientes [q_1, ..., q_s]
    - r: el residuo (resto)
    """
    # Inicializamos variables 
    s = len(F)
    q = [0] * s  # q_1, ..., q_s
    r = 0
    p = f
    
    while p != 0:
        division_occurred = False # Bandera para saber si dividimos en este paso
        i = 0
        
        while i < s and not division_occurred:
            fi = F[i]
            # Revisar si el término líder de fi divide al de p
            if fi != 0 and fi.lt().divides(p.lt()):
                # Paso de División: LT(f_i) divide a LT(p)
                quotient_term = p.lt() // fi.lt()  # División exacta en el anillo
                q[i] += quotient_term
                p -= quotient_term * fi
                division_occurred = True
            else:
                i += 1
        
        if not division_occurred:
            # Paso del Residuo: el término líder de p va al residuo
            r += p.lt()
            p -= p.lt()
    
    return q, r


def S_polynomial(f, g):
    """
    Calcula el S-polinomio de f y g como se define en la Def 4 del §6.
    
    S(f, g) = (x^gamma / LT(f)) * f - (x^gamma / LT(g)) * g
    donde x^gamma = mcm(LM(f), LM(g))
    """
    R = f.parent()
    
    # Sacamos términos líderes (LT), monomios líderes (LM) y coeficientes líderes (LC)
    LT_f = f.lt()
    LT_g = g.lt()
    LM_f = f.lm()
    LM_g = g.lm()
    LC_f = f.lc()
    LC_g = g.lc()
    
    # Calcular el MCM (gamma) de los monomios líderes
    gamma = lcm_monomials(LM_f, LM_g)
    
    # Calcular las partes monomiales: x^gamma / LM(f) y x^gamma / LM(g)
    monom1 = gamma // LM_f
    monom2 = gamma // LM_g
    
    # Aplicar la fórmula del libro para el S-poly
    term1 = (monom1 * f) / LC_f
    term2 = (monom2 * g) / LC_g
    
    return term1 - term2


def lcm_monomials(m1, m2):
    """
    Calcula el Mínimo Común Múltiplo (LCM) de dos monomios.
    Para x^alpha y x^beta, el LCM es x^gamma donde gamma_i = max(alpha_i, beta_i).
    """
    exponents1 = m1.exponents()[0]
    exponents2 = m2.exponents()[0]
    
    # Tomar el máximo componente a componente como dice el texto
    lcm_exponents = [max(e1, e2) for e1, e2 in zip(exponents1, exponents2)]
    
    return m1.parent().monomial(*lcm_exponents)


In [3]:
def poly_to_Macaulay(F, D):
    R = F[0].parent()
    order = R.term_order()
    
    # Paso 1: Construimos todos los polinomios u*f
    UF = []
    for f in F:
        df = f.degree()
        for d in range(D - df + 1):
            for u in R.monomials_of_degree(d):
                UF.append(u * f)

    # Paso 2: Construimos  el conjunto U de monomios que aparecen
    U = set()
    for g in UF:
        for m in g.monomials():
            U.add(m)

    # Paso 3: Ordenamos U con el orden monomial del anillo
    U = sorted(U, reverse=True)
    #print(U)

    # Paso 4: Construimos la matriz de Macaulay
    M = []
    for g in UF:
        fila = [g.monomial_coefficient(u) for u in U]
        M.append(fila)

    return Matrix(M), U


R.<x,y> = PolynomialRing(QQ, order='lex')

f1 = x^2 + y^2 +5*x^2*y
f2 = x*y + 1
F = [f1, f2]

D = 3

# Llamado para construir la matriz de Macaulay
M, U = poly_to_Macaulay(F, D)

print(M)

[5 1 0 0 0 1 0 0]
[0 0 0 1 0 0 0 1]
[0 0 1 0 0 0 1 0]
[1 0 0 0 1 0 0 0]


In [4]:
def macaulay_to_Poly(M, U):
    
    R = U[0].parent()
    polinomios_base = []
    
    for fila in M.rows():
        # Sage: M.rows() ya maneja la conversión de la matriz a una secuencia de filas
        
        if not fila.is_zero():
            # Construir el polinomio sumando el producto (coeficiente * monomio)
            # zip(fila, U) empareja el coeficiente de la columna j con el monomio U[j]
            p = sum(c * m for c, m in zip(fila, U))
            
            # Es buena práctica asegurar que el coeficiente líder sea 1 si M es RREF
            if p.lc() != 0 and p.lc() != 1:
                 p = p / p.lc()
                 
            polinomios_base.append(p)
            
    # Devolvemos solo los polinomios reconstruidos
    return polinomios_base
    """
    F = [0]*len(M.transpose()[0])
    
    for i in range(0, len(M.transpose()[0])):
        for j in range(0, len(M[0])):
            F[i] += M[i][j]*U[j] 
            
    return F
    """
        
M_echelon = M.echelon_form()
F = macaulay_to_Poly(M_echelon,U)
is_Groebner(F)
print(F)

[x^2*y + x, x^2 - 5*x + y^2, x*y^2 + y, x*y + 1]


In [5]:
def find_Grobner_Basis(F):
    grados = [f.degree() for f in F]
    D = max(grados)
    basis = F
    
    entered_loop = False
    
    while is_Groebner(basis) == False: 
        entered_loop = True
        M, U = poly_to_Macaulay(F,D)
        M_echelon = M.echelon_form()
        basis = macaulay_to_Poly(M_echelon, U)
        D += 1
        
    if entered_loop:    
        return basis, D-1
    else:
        return basis, D


R.<x,y,z> = PolynomialRing(GF(7), order='deglex')

f1 = x^2 + y^2 +5*x^2*y + x*y*z
f2 = x*y + z
F = [f1, f2]

basis, D = find_Grobner_Basis(F)
for i in range(0, len(basis)):
    print(basis[i])
print(f"D: {D}")

grados = [f.degree() for f in F]
Dmax = max(grados)
print(Dmax)

x^3*y - 2*x*z^2 - y^2*z + z^3
x^2*y^2 - z^2
x^2*y*z + x*z^2
x*y^3 + y^2*z
x*y^2*z + y*z^2
x*y*z^2 + z^3
x^3 + 2*x*z^2 - 2*y^2*z + 2*z^3 - y*z
x^2*y + x*z
x^2*z + 2*x*z^2 + y^2*z - z^3
x*y^2 + y*z
x*y*z + z^2
y^3 - y*z^2 - x*z - 2*z^2
x^2 + 2*x*z + y^2 - z^2
x*y + z
D: 4
3


In [28]:
import pandas as pd
import time

R.<x,y,z> = PolynomialRing(GF(673), order='deglex')

Polinomios_lista = []
D_lista = []
bases_lista = []

start = time.
for i in range(0,1000):
    
    f = R.random_element(degree=2)
    g = R.random_element(degree=2)
    
    F = [f,g]
    basis, D = find_Grobner_Basis(F)
    
    Polinomios_lista.append(F)
    D_lista.append(D)
    bases_lista.append(basis)
    


In [32]:


resultados = pd.DataFrame({"Polinomios":Polinomios_lista, "D":D_lista, "bases": bases_lista})
resultados.to_csv("grupo_GF(673).csv")

In [27]:
bases_lista[-1]

[[...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],
 [...],


In [ ]:
R.<x,y,z,w> = PolynomialRing(QQ, order='deglex')

f = R.random_element()
g = R.random_element()
h = R.random_element()

F = [f,g,h]

_, D = find_Grobner_Basis(F)

D